In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

2023-06-09 21:04:46.359405: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-09 21:04:46.403535: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-09 21:04:46.404722: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-09 21:04:47.152109: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# Load the TFLite model
interpreter = tf.lite.Interpreter(model_path='/home/usern/Desktop/ai/exportedMMobil.tflite')
interpreter.allocate_tensors()

INFO: Created TensorFlow Lite delegate for select TF ops.
INFO: TfLiteFlexDelegate delegate: 14 nodes delegated out of 187 nodes with 1 partitions.

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [ ]:
my_signature = interpreter.get_signature_runner()

In [4]:
# Get input and output details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
print(input_details)

[{'name': 'serving_default_duration:0', 'index': 0, 'shape': array([1, 1], dtype=int32), 'shape_signature': array([-1,  1], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}, {'name': 'serving_default_own_telephone:0', 'index': 1, 'shape': array([1, 1], dtype=int32), 'shape_signature': array([-1,  1], dtype=int32), 'dtype': <class 'numpy.bytes_'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}, {'name': 'serving_default_marriage:0', 'index': 2, 'shape': array([1, 1], dtype=int32), 'shape_signature': array([-1,  1], dtype=int32), 'dtype': <class 'numpy.bytes_'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], 

In [ ]:
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
  df = dataframe.copy()
  df = {key: value[:,tf.newaxis] for key, value in dataframe.items()}
  ds = tf.data.Dataset.from_tensor_slices((dict(df)))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(dataframe))
  ds = ds.batch(batch_size)
  ds = ds.prefetch(batch_size)
  
  return ds

In [ ]:
data = [['<0', 6.0, 'critical/other existing credit', 'radio/tv', 1169.0, 'no known savings', '>=7', 4.0, 'none', 4.0, 'real estate', 67.0, 'none', 'own', 2.0, 'skilled', 1.0, 'yes', 'yes','male','single']]
#         ['0<=X<200', 48.0, 'existing paid', 'radio/tv', 5951.0, '<100', '1<=X<4', 2.0, 'none', 2.0, 'real estate', 22.0, 'none', 'own', 1.0, 'skilled', 1.0, 'none', 'yes', 'female','div/dep/mar'],
#         ['no checking', 12.0, 'existing paid', 'radio/tv', 3059.0, '>=1000', '4<=X<7', 2.0, 'none', 4.0, 'real estate', 61.0, 'none', 'own', 1.0, 'unskilled resident', 1.0, 'none', 'yes', 'male', 'div/sep'],
# ['0<=X<200', 30.0, 'critical/other existing credit', 'new car', 5234.0, '<100', 'unemployed', 4.0, 'none', 2.0, 'car', 28.0, 'none', 'own', 2.0, 'high qualif/self emp/mgmt', 1.0, 'none', 'yes', 'male', 'mar/wid']]


columns = ['checking_status', 'duration', 'credit_history', 'purpose',
           'credit_amount', 'savings_status', 'employment',
           'installment_commitment', 'other_parties', 'residence_since',
           'property_magnitude', 'age', 'other_payment_plans', 'housing',
           'existing_credits', 'job', 'num_dependents', 'own_telephone',
           'foreign_worker', 'sex', 'marriage']

# Create the pandas DataFrame with column name is provided explicitly
df = pd.DataFrame(data, columns=columns)

In [ ]:
ds=df_to_dataset(df,False,len(df))

In [ ]:
# Perform inference on the test data
interpreter.set_tensor(input_details[0]['index'],ds)
interpreter.invoke()

In [ ]:
print("== Input details ==")
print("name:", input_details[0]['name'])
print("shape:", input_details[0]['shape'])
print("type:", input_details[0]['dtype'])

In [ ]:
print("\n== Output details ==")
print("name:", output_details[0]['name'])
print("shape:", output_details[0]['shape'])
print("type:", output_details[0]['dtype'])

In [ ]:
input_shape = input_details[0]['shape']

In [ ]:
output = my_signature(x=df.values)

In [ ]:
print(interpreter.get_signature_list())